In [ ]:
#pip install dash
#pip install dash-leaflet

Useful links:
https://www.dash-leaflet.com/components/controls/edit_control

Definition of useful libraries

In [3]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_leaflet as dl
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import mapping
import leafmap as leafmap
from shapely.geometry import shape
from dash_extensions.javascript import assign

Definition of useful functions

In [4]:
# read_response checks if the response of the server is valid JSON

def read_response(t):
    try:
        data = t.json() #This will convert the response to a json object
        return data
    except requests.exceptions.JSONDecodeError:
        print("Risposta non valida JSON!")
        print("Contenuto della risposta:", t.text)
        data = None 

# get_measurement_unit returns the measurement unit of a given pollutant

def get_measurement_unit(pollutant):
    t=requests.post(url="http://127.0.0.1:5000/api/units", json={"var_pollutant": pollutant}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data[0]

# get_province_shape returns a GeoDataFrame containing the geometries of the provinces
   
def get_province_shape():
    t=requests.get(url="http://127.0.0.1:5000/api/province_shape")
    data = read_response(t)
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry_province'] = gdf['geometry_province'].apply(wkt.loads)
    gdf.set_geometry('geometry_province', inplace=True)
    return gdf

# list_sensors_and_pollutants gets the list of sensor and relative pollutants for a given station

def list_sensors_and_pollutants(list_stations):
    t=requests.post(url="http://127.0.0.1:5000/api/sensors_and_pollutants", json={"var_id_stazione": list_stations}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data

# get_station_location returns a GeoDataFrame containing the points of the stations

def get_station_location():
    t=requests.get(url="http://127.0.0.1:5000/api/station_location")
    data = read_response(t)
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    gdf.set_geometry('geometry', inplace=True)
    return gdf

############################################# FUNCTION TO POPULATE THE DROPDOWN #############################################

# pollutant_dropdown populates the dropdown with the pollutatnts that have at least one recorded value

def pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list

# all_pollutant_dropdown populates the dropdown with all the pollutants, even those that have no recorded values

def all_pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/all_pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list

############################################# FUNCTION FOR MAP VISUALIZATION #############################################

# df_to_dash_table converts a DataFrame to a Dash HTML table for display in the popup of EU_DV_1

def df_to_dash_table(df):
    return html.Table([
        html.Thead(html.Tr([html.Th(col) for col in df.columns])),
        html.Tbody([
            html.Tr([html.Td(df.iloc[i][col]) for col in df.columns]) for i in range(len(df))
        ])
    ], style={"maxWidth": "300px", "fontSize": "12px"})

Definition of variables for visualization

In [5]:
# prov is a GeoDataFrame containing the geometries of the provinces
prov=get_province_shape()

# geojson is a dictionary containing the geometry of the provinces in GeoJSON format
geojson = mapping(prov.geometry_province)

# the two following variables are used to create a green and a red icon marker, later used in the map

#https://github.com/pointhi/leaflet-color-markers
custom_icon_green = dict(
    iconUrl= 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-green.png',
    shadowUrl= 'https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.7/images/marker-shadow.png',
    iconSize= [25, 41],
    iconAnchor= [12, 41],
    popupAnchor= [1, -34],
    shadowSize= [41, 41]
)
custom_icon_red = dict(
    iconUrl= 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-red.png',
    shadowUrl= 'https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.7/images/marker-shadow.png',
    iconSize= [25, 41],
    iconAnchor= [12, 41],
    popupAnchor= [1, -34],
    shadowSize= [41, 41]
)

# How to render geojson.
point_to_layer = assign("""function(feature, latlng, context){
    const p = feature.properties;
    if(p.type === 'circlemarker'){return L.circleMarker(latlng, radius=p._radius)}
    if(p.type === 'circle'){return L.circle(latlng, radius=p._mRadius)}
    return L.marker(latlng);
}""")


In [ ]:
# Initialize the Dash app
app = Dash(__name__)

# Define the layout
app.layout = html.Div([

    html.Div([
    html.Div([
        html.H1('Bugs_project: Air quality analysis'),
        html.P('This is a project developed by Davide Galluzzo, Silvia Macchi, Giovanni Pasut and Sveva Zanetti for the course of Software Engineering for Geoinformatics 2024/2025'),
        html.P('The project consists in an interactive dashboard that allows the user to visualize and analyse air quality and sensor data provided by Regione Lombardia'),
    ], style={'width': '75%', 'padding': 0}),

    html.Div([
        html.Img(src='/assets/logo.png', style={'width': '100%', 'maxWidth': '200px'})
    ], style={
        'width': '25%',
        'padding': 10,
        'textAlign': 'right',
        'alignSelf': 'flex-start'
    }),
    ], style={
        'display': 'flex',
        'justifyContent': 'space-between',
        'alignItems': 'flex-start',
        'margin': 2,
        'padding': 20
    }),
    
    ############################################# User selection #############################################
    html.Div([

    html.H2('User selection'),
    html.P(['According to your knowledge about air pollution, select which type of user represents you. ',
           html.Br(),
        'Expert user have acces to more advanced analysis, allowing them to define some thresholds and areas of interest']),
    html.Div(
        dcc.RadioItems(
            id='user-selection',
            options=[
                {'label': 'Normal User', 'value': 1},
                {'label': 'Expert User', 'value': 2}
            ],
            value=0,
            labelStyle={'display': 'inline-block', 'margin-right': '20px', 'padding':'20px','fontSize': '20px'}
        ),)
    ],
    id='user', hidden=False, className='div_border'style={'textAlign': 'center'}),

    ############################################# DV_5 #############################################
    html.Div([
        html.Div([
        html.H2('DV_5 Single Sensor Time Series'),
        html.P('Start by selecting a pollutant, then select one among all the sensors that measures that pollutant, and then visualize in the graph the time series of the monthly average of the data measured by that sensor'),

        #Dropdown to select the pollutant
            dcc.Dropdown(
            id='pollutant-dropdown_dv_5',
            options=pollutant_dropdown(),
            placeholder="Select a pollutant",
            style={'margin-top':20}
        ),

        #Dropdown to select the sensor
            dcc.Dropdown(
            id='sensor-dropdown_dv_5',
            placeholder="Select a sensor",
            style={'margin-top':20}
        ),

        #Button to activate the graph
        html.Div([
            html.Button('Visualize/update time series', id='time_series_dv_5', n_clicks=0, className='button')
        ], style={'textAlign': 'center'}),

        #Visualiza the graph
        # Loading wrapper for the map
        dcc.Loading(
            type="dot",  
            children=dcc.Graph(id='time_series_single_sensor_dv_5'),
        )
        
        ], className='div_border'),

        ############################################# DV_6 #############################################

        html.Div([
        html.H2('DV_6 Location of all the sensors of a selected pollutant'),
        html.P('Choose a pollutant, and then visualize on a map the location af all the sensors of that pollutant. The sensor will appear red if it\'s still active, and will appear gray if it is not active anymore. Clicking on the icon of a sensor the user can visualiza the id of the sensor, the name of the station in which the sensor is located and date in which the sensor was deactivated, if it is not active anymore'),
        
        #Dropdown to select the pollutant
            dcc.Dropdown(
            id='all-pollutant-dropdown_dv_6',
            options=all_pollutant_dropdown(),
            placeholder="Select a pollutant",
            style={'margin-top':20}
        ),

        #Button to activate the map
        html.Div([
            html.Button('Visualize/update map', id='button_dv_6', n_clicks=0, className='button')
        ], style={'textAlign': 'center'}),

        #Visualization of the map
        dcc.Loading(
            type="dot",  
            children=dl.Map(
                children=[
                    dl.TileLayer(),  # base layer
                    dl.LayersControl([dl.Overlay(
                            dl.GeoJSON(
                                data=geojson,
                                id="province_dv_6",
                                zoomToBounds=True,
                                options=dict(style=dict(color="blue", weight=1))
                            ),
                            name="Province",
                            checked=True
                        ),
                        dl.Overlay(dl.LayerGroup(id="active_dv_6", children=[]), name="Active sensors", checked=True),
                        dl.Overlay(dl.LayerGroup(id="inactive_dv_6", children=[]), name="Inactive sensors", checked=True),],id="lc_dv_6")
                ],
                center=[45.64, 9.60],
                zoom=8,
                style={"height": "50vh"}
            )
        )
        
        ], className='div_border'),

        ############################################# User upgrade #############################################
        html.Div([
            html.Div([
                html.P('Upgrade to expert user and unlock two more analysis'),
                html.Button('Click to upgrade', id='button_upgrade', n_clicks=0, style={'margin-top': 20, 'margin-bottom':20, 'padding': '6px 12px', 'fontSize': '15px',})
            ], style={'textAlign': 'center'}),
        ], id='user_upgrade', hidden=False, className='div_border'),
    ], id='normal_user'),

    ############################################# EU_DV_1 #############################################
    html.Div([
        html.Div([
        html.H2('EU_DV_1 Expert user'),
        html.P('In this section an expert user can select a region of interest on the map using the buttons on the left. Once the region of interest has been selected the user can visualize on the map the stations inside the selected area. The station will appear red if it contains at least one active sensor, otherwise it will be gray. By clicking on the icon of a station the user can visualize which sensors are inside the station, the pollutant that they measure and the date in which they have been deactivated, if they are not active anymore'),
            
        # Display the map in which the user can select a region of interest
        dl.Map(
            children=[
            dl.TileLayer(),  # base layer
            dl.FeatureGroup([
                dl.EditControl(id="edit_control_eu_dv_1",draw={
                    "circle": False,         
                    "circlemarker": False,   
                    "rectangle": True,
                    "polygon": True,
                    "marker": False,
                    "polyline": False,
                    },),
            ]),
            dl.LayersControl(dl.Overlay(dl.GeoJSON(
                        data=geojson,
                        id="lombardia_eu_dv_1",
                        zoomToBounds=True,
                        #interactive=False,
                        options=dict(style=dict(color="blue", weight=1))
                    ), name="Lombardia",checked=True ))],
            center=[45.64, 9.60],
            zoom=8,
            style={"height": "50vh"}
        ),
        
        #Button the reset the selection of the user
        html.Div([
            html.Button("Remove -> Clear all", id="clear_all_eu_dv_1", className='button'),
        ], style={'textAlign': 'center'}),
        

        html.P("These are the stations inside the selected region of interes, if the region is empty it means that no stations are inside the select area"),

        # Display the stations inside the region of interest on the new map
        dl.Map(
            children=[
                dl.TileLayer(),  # base layer
                dl.GeoJSON(id="geojson_eu_dv_1", pointToLayer=point_to_layer, zoomToBounds=True),
                dl.LayersControl([dl.Overlay(
                        dl.GeoJSON(
                            data=geojson,
                            id="province_eu_dv_1",
                            zoomToBounds=True,
                            options=dict(style=dict(color="blue", weight=1))
                        ),
                        name="Province",
                        checked=True
                    ),
                    dl.Overlay(dl.LayerGroup(id="active_eu_dv_1", children=[]), name="Active sensors", checked=True),
                    dl.Overlay(dl.LayerGroup(id="inactive_eu_dv_1", children=[]), name="Inactive sensors", checked=True),],id="lc_eu_dv_1")
            ],
            style={"height": "50vh"},
            center=[45.64, 9.60],
            zoom=7,id="final"),
        ], className='div_border'),
    ], id='expert_user'),
])

############################################# Callbacks user #############################################

@app.callback(
    [Output('normal_user', 'hidden'),
    Output('expert_user', 'hidden'),
    Output('user', 'hidden'),
    Output('user_upgrade','hidden')
    ],
    [Input('user-selection','value'),
     Input('button_upgrade','n_clicks')]
)
def user_selection(selected_user,n_clicks):
    if selected_user == 0:
        return [True, True, False, True]
    elif selected_user == 1 and n_clicks == 0:
        return [False, True, True, False]  
    else:
        return [False, False, True, True]
    
############################################# Callbacks DV_5 #############################################

#Callback to get the sensor list after the user selects a pollutant
@app.callback(
    Output('sensor-dropdown_dv_5', 'options'),
    Input('pollutant-dropdown_dv_5','value')
)
def sensor_dropdown(selected_pollutant):
    if not selected_pollutant:
        options = []
        return options
    t=requests.post(url="http://127.0.0.1:5000/api/sensor", json={"var_pollutant": selected_pollutant})
    list_sensors = read_response(t)
    options = [{'label': f'{sensor['nome_stazione']} - {sensor['id_sensore']}', 'value': sensor['id_sensore']} for sensor in list_sensors]
    return options

#Callback to print the graph of of requirement 5: time series of a single sensor
@app.callback(
    Output('time_series_single_sensor_dv_5','figure'),
    Input('time_series_dv_5', 'n_clicks'),
    State('pollutant-dropdown_dv_5','value'),
    State('sensor-dropdown_dv_5','value')
)
def time_series(n_clicks, selected_pollutant, sensor_id):
    if n_clicks > 0:
        if not sensor_id:
            return px.line().update_layout(
            annotations=[dict(
                    text="Please select a pollutant and a sensor to visualize the plot",
                    xref="paper", yref="paper",
                    showarrow=False,
                    font=dict(size=14, color="white"),
                    bgcolor="red",
                    borderpad=5)])
        data = {"var_sensor_id": sensor_id, "var_pollutant": selected_pollutant} #Right now data is a dictionary
        # send the request to the server, we use post because we are sending data to the server
        t=requests.post(url="http://127.0.0.1:5000/api/DV_5", json=data) #json= data will convert the dictionary to a json object and send it to the server
        data = read_response(t) 
        df = pd.DataFrame(data)
        if df.empty:
            return px.line(title='No data available for the selected sensor and pollutant')
        df['month'] = pd.to_datetime(df['month'], errors='coerce') #keeps just year and month for the date
        df['month'] = df['month'].dt.strftime('%b %Y')
        unit=get_measurement_unit(selected_pollutant) #gets the unit of measurement for a pollutant
        line_chart = px.line(df, x='month', y='monthly_avg', title=f'Monthly average for {selected_pollutant} of sensor {sensor_id} in {unit}')
        return line_chart
    return px.line()

############################################# Callbacks DV_6 #############################################

#Callback to print the map of requirement 6
@app.callback(
    [Output('active_dv_6', 'children'),
     Output('inactive_dv_6','children')],
    Input('button_dv_6', 'n_clicks'),
    State('all-pollutant-dropdown_dv_6','value')
)
def map_sensors_location(n_clicks, selected_pollutant):

    if n_clicks == 0:
      return [[],[]]
    
    if n_clicks > 0 and not selected_pollutant:
        return [dl.Popup(
                    position=[45.64, 9.60],  # Central position
                    children=html.Div(
                        "Select a pollutant to visualize the map",
                        style={"backgroundColor": "red", "color": "white", "padding": "10px", "borderRadius": "5px"}
                    ))
                ,[]
            ]

    if n_clicks > 0:
        #if not selected_pollutant:
         #   return [[],[]]
    
        #get the data
        data = {"var_pollutant" : selected_pollutant}

        # send the request to the server, we use post because we are sending data to the server
        t=requests.post(url="http://127.0.0.1:5000/api/DV_6", json=data) #json= data will convert the dictionary to a json object and send it to the server

        data = read_response(t)
        gdf = gpd.GeoDataFrame(data)
        gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
        gdf.set_geometry('geometry', inplace=True)
        
        #create the markers
        active = gdf[gdf['data_stop'].isnull()]
        inactive = gdf[gdf['data_stop'].notnull()]
        inactive['data_stop'] = pd.to_datetime(inactive['data_stop'], errors='coerce') #keeps just year and month for the date
        inactive['data_stop'] = inactive['data_stop'].dt.strftime('%b %Y')

        # Crea Marker attivi
        active_markers = [
            dl.Marker(
                position=[row.geometry.y, row.geometry.x],
                children=dl.Popup([
                    html.B(f"{row['id_sensore']} - {row['nome_stazione']}"),
                    html.Br(),
                    "Active"
                ]),
                icon=custom_icon_green
            ) for _, row in active.iterrows()
        ]

        # Marker inattivi
        inactive_markers = [
            dl.Marker(
                position=[row.geometry.y, row.geometry.x],
                children=dl.Popup([
                    html.B(f"{row['id_sensore']} - {row['nome_stazione']}"),
                    html.Br(),
                    f"Deactivated on {row['data_stop']}"
                ]),
                icon=custom_icon_red
            ) for _, row in inactive.iterrows()
        ]

        return [active_markers, inactive_markers]
    return [[],[]]

############################################# Callbacks EU_DV_1 #############################################

# Callback to get the region of interest choosen by the user
@app.callback(
    Output("geojson_eu_dv_1", "data"),
    Input("edit_control_eu_dv_1", "geojson"))
def mirror(x):
    return x

# Callback to delete the region of interest choosen by the user
@app.callback(
    Output("edit_control_eu_dv_1", "editToolbar"), 
    Input("clear_all_eu_dv_1", "n_clicks"))
def trigger_action(n_clicks):
    return dict(mode="remove", action="clear all", n_clicks=n_clicks)  # include n_click to ensure prop changes

# Callback to populate the region of interest choosen by the user
@app.callback(
    [Output("active_eu_dv_1", "children"),
     Output("inactive_eu_dv_1","children")],
    Input("geojson_eu_dv_1", "data")
)
def get_station_inside_geometry(ROI):

    # If no ROI is selected, return empty lists
    if not ROI or "features" not in ROI or len(ROI["features"]) == 0:
        return [[], []]
    
    # Find the stations inside the ROI
    polygon_ROI = shape(ROI["features"][0]["geometry"])
    station = get_station_location()
    intersection = polygon_ROI.intersection(station.geometry)
    
    # Filters the stations that are inside the ROI
    new = station.copy()
    valid_indices = [i for i, geom in enumerate(intersection) if not geom.is_empty]
    new = new.iloc[valid_indices]

    if new.empty:
        return [[], []]

    # Gets all the sensors inside the filtered stations
    lista = new.id_stazione.tolist() #Converto to a list the stations inside the area
    list_sensors_inside_area=list_sensors_and_pollutants(lista) #seach for all sensors inside the area
    df = pd.DataFrame(list_sensors_inside_area) #created a df with all the sensors inside the area
    df['data_stop'] = pd.to_datetime(df['data_stop'], errors='coerce')
    df['data_stop'] = df['data_stop'].dt.strftime('%b %Y')

    # Create markers for the stations
    active_markers = []
    inactive_markers = []
    for index, row in new.iterrows():
        filtro = df[df['id_stazione'] == row['id_stazione'] ]
        filtro = filtro.drop('id_stazione', axis=1)
        if filtro["data_stop"].isna().any():
            # Crea Marker attivi
            active_markers.append(
                dl.Marker(
                    position=[row.geometry.y, row.geometry.x],
                    children=dl.Popup([
                        html.B(f"Dati rilevati nella stazione {row['id_stazione']} - {row['nome_stazione']}"),
                        html.Br(),
                        df_to_dash_table(filtro),
                    ]),
                    icon=custom_icon_green
                )
            )
        else:
            inactive_markers.append(
                dl.Marker(
                    position=[row.geometry.y, row.geometry.x],
                    children=dl.Popup([
                        html.B(f"Dati rilevati nella stazione {row['id_stazione']} - {row['nome_stazione']}"),
                        html.Br(),
                        df_to_dash_table(filtro),
                    ]),
                    icon=custom_icon_red
                )
            )
    return [active_markers, inactive_markers]

# Run the app in Jupyter notebook
print("Starting Dashboard...")
#app.run_server(port=8089)
app.run(port=8089, debug=True)

Starting Dashboard...
